CNN version 1.0
=============
Using the preprocessed Data
------------

Previously in `Preprocessing.ipynb`, we filtered various forms of inconsistencies from the data and made it appropriate for using the model.

The goal of this noteook is to run a simple CNN here.

In [107]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# Task 1: Load the data from the pickle file

In [108]:
'''Set the constant paths here'''
root = "../Data"
path = "../Data/Resized_PreData(100x100)"
ori_path = "../Data/PreData/"
csv_path = "../Data/train_info.csv"

Retrieve the saved data from the pickle file

In [109]:
pickle_files_names = ("PreData.pickle",)

unFormatted_Data = []


for Dfile in pickle_files_names:
    with open(os.path.join(root, Dfile), "rb") as f:
        unFormatted_Data.append(pickle.load(f))

In [110]:
len(unFormatted_Data)

1

In [111]:
# recollect the fragmented data

labels_mappings = unFormatted_Data[0]['label_mapping']
print(label_mappings)

train_dataSet = unFormatted_Data[0]['train_data']
train_labels  = unFormatted_Data[0]['train_labels']

cv_dataSet = unFormatted_Data[0]['cv_data']
cv_labels = unFormatted_Data[0]['cv_labels']

test_dataSet = unFormatted_Data[0]['test_data']
test_labels  = unFormatted_Data[0]['test_labels']

print(train_dataSet.shape, cv_dataSet.shape, test_dataSet.shape)
print(train_labels.shape, cv_labels.shape, test_labels.shape)

{'symbolic painting': 17, 'caricature': 2, 'sketch and study': 15, 'mythological painting': 8, 'capriccio': 1, 'genre painting': 4, 'history painting': 5, 'literary painting': 7, 'illustration': 6, 'self-portrait': 14, 'pastorale': 10, 'nude painting (nu)': 9, 'allegorical painting': 0, 'design': 3, 'still life': 16, 'poster': 12, 'portrait': 11, 'religious painting': 13}
(180, 100, 100, 3) (26, 100, 100, 3) (52, 100, 100, 3)
(180, 20) (26, 20) (52, 20)


# Task 2: Spawn the CNN 

In [112]:
# The accuracy function to calculate the accuracy of the classifier

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

define the constants required hence forth:

In [129]:
batch_size = 16
patch_size = 5
depth = 16 # number of filters in a layer
num_hidden = 128

image_size = 100
num_labels = 20
num_channels = 3 # rgb images

### The CNN tensorflow graph 

In [130]:
graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
    tf_valid_dataset = tf.constant(cv_dataSet)
    tf_test_dataset = tf.constant(test_dataSet)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    validation_prediction = tf.nn.softmax(model(tf_valid_dataset))

In [131]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
  
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataSet[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
    
    print('Cross_Validation accuracy: %.1f%%' % accuracy(validation_prediction.eval(), cv_labels))

Initialized
Minibatch loss at step 0: 3.407605
Minibatch accuracy: 18.8%
Minibatch loss at step 50: 2.240751
Minibatch accuracy: 50.0%
Minibatch loss at step 100: 1.460077
Minibatch accuracy: 56.2%
Minibatch loss at step 150: 2.193918
Minibatch accuracy: 43.8%
Minibatch loss at step 200: 1.938498
Minibatch accuracy: 43.8%
Minibatch loss at step 250: 1.727551
Minibatch accuracy: 50.0%
Minibatch loss at step 300: 0.483191
Minibatch accuracy: 81.2%
Minibatch loss at step 350: 0.650792
Minibatch accuracy: 75.0%
Minibatch loss at step 400: 0.285093
Minibatch accuracy: 87.5%
Minibatch loss at step 450: 0.177352
Minibatch accuracy: 100.0%
Minibatch loss at step 500: 0.120597
Minibatch accuracy: 100.0%
Minibatch loss at step 550: 0.359475
Minibatch accuracy: 87.5%
Minibatch loss at step 600: 0.010564
Minibatch accuracy: 100.0%
Minibatch loss at step 650: 0.004155
Minibatch accuracy: 100.0%
Minibatch loss at step 700: 0.002463
Minibatch accuracy: 100.0%
Minibatch loss at step 750: 0.011141
Mini